# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
rabieoudghiri_plantvillage_split_70_15_15_path = kagglehub.dataset_download('rabieoudghiri/plantvillage-split-70-15-15')

print('Data source import complete.')


In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2,
    shear_range=0.2
)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
import os
import kagglehub
rabieoudghiri_plantvillage_split_70_15_15_path = kagglehub.dataset_download('rabieoudghiri/plantvillage-split-70-15-15')
base_data_path = rabieoudghiri_plantvillage_split_70_15_15_path
train_dir = os.path.join(base_data_path, 'plantvillage_split', 'train')
val_dir = os.path.join(base_data_path, 'plantvillage_split', 'valid')
test_dir = os.path.join(base_data_path, 'plantvillage_split', 'test')

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224,224), batch_size=64, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224,224), batch_size=64, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224,224), batch_size=64, class_mode='categorical')

Found 37997 images belonging to 38 classes.
Found 8129 images belonging to 38 classes.
Found 8180 images belonging to 38 classes.


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

from tensorflow.keras.optimizers import SGD

model.compile(
    optimizer=SGD(learning_rate=1e-3, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    callbacks=[early_stop]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
594/594 ━━━━━━━━━━━━━━━━━━━━ 163s 216ms/step - accuracy: 0.3944 - loss: 2.5210 - val_accuracy: 0.8420 - val_loss: 0.6806
Epoch 2/15
594/594 ━━━━━━━━━━━━━━━━━━━━ 86s 145ms/step - accuracy: 0.8171 - loss: 0.7629 - val_accuracy: 0.9151 - val_loss: 0.3548
Epoch 3/15
594/594 ━━━━━━━━━━━━━━━━━━━━ 87s 146ms/step - accuracy: 0.8749 - loss: 0.4709 - val_accuracy: 0.9366 - val_loss: 0.2461
Epoch 4/15
594/594 ━━━━━━━━━━━━━━━━━━━━ 87s 146ms/step - accuracy: 0.9038 - loss: 0.3447 - val_accuracy: 0.9487 - val_loss: 0.1936
Epoch 5/15
594/594 ━━━━━━━━━━━━━━━━━━━━ 87s 146ms/step - accuracy: 0.9233 - loss: 0.2772 - val_accuracy: 0.9573 - val_loss: 0.1622
Epoch 6/15
594/594 ━━━━━━━━━━━━━━━━━━━━ 85s 144ms/step - accuracy: 0.9300 - loss: 0.2402 - val_accuracy: 0.9615 - val_loss: 0.1420
Epoch 7/15
594/594 ━━━━━━━━━━━━━━━━━━━━ 85s 143ms/step - accuracy: 0.9385 - loss: 0.2116 - val_accuracy: 0.9631 - val_loss: 0.1288
Epoch 8/15
594/594 ━━━━━━━━━━━━━━━━━━━━ 87s 146ms/step - accuracy: 0.9449 - loss: 

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    "best_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max"
)
callbacks=[early_stop, checkpoint]

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")

128/128 ━━━━━━━━━━━━━━━━━━━━ 27s 211ms/step - accuracy: 0.9700 - loss: 0.0923
Test accuracy: 0.9691


In [ ]:
model.save('/content/plant_disease_efficientnetb0V2.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save model to Google Drive
model.save('/content/drive/MyDrive/plant_disease_efficientnetb0V2.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
